# 中級：クラスタリング（K-Means）によるユーザーセグメント抽出

この章では、ラベル（購入意向など）を使わずに、ユーザーの特徴（属性＋感情カテゴリ）をもとに  
似たユーザー群をクラスタリングして、セグメントを発見する分析をします。

ゴールはこんなこと：
- ユーザー群ごとのプロファイル（年齢構成、カテゴリ傾向、満足度の傾向など）を把握  
- 各クラスタの特徴をマーケティング仮説に結びつける  
- モデルを使って新規ユーザーのクラスタを予測できるようにする  

使う手法：SparkML の `KMeans` クラス（`pyspark.ml.clustering.KMeans`）  [oai_citation:0‡api-docs.databricks.com](https://api-docs.databricks.com/python/pyspark/latest/api/pyspark.ml.clustering.KMeans.html?utm_source=chatgpt.com)

In [0]:
CATALOG = "hiroshi"
SCHEMA  = "survey_analysis_simple"

# UCオブジェクト作成
spark.sql(f"USE CATALOG {CATALOG}")
spark.sql(f"USE {SCHEMA}")

GOLD_TABLE_NAME = "gold_survey_responses_final"

## 必要モジュール読み込みとデータ取得

このセルでは、クラスタリングに必要な SparkML モジュールをインポートし、  
GOLD テーブルを取得してクラスタリング対象データを準備します。

In [0]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
from pyspark.sql import functions as F

In [0]:
gold = spark.table(GOLD_TABLE_NAME)  # ← 実際の GOLD テーブル名
display(gold.limit(10))
gold.printSchema()

## クラスタリングに使う特徴量列の選定とカテゴリ変換準備

このセルでは、クラスタリングに使いたい列（カテゴリ変数、数値変数、感情カテゴリ列など）を選び、  
カテゴリ変数を数値ベクトルに変換するための StringIndexer + OneHotEncoder を定義します。

In [0]:
categorical_cols = ["age_group", "gender", "region",
                    "positive_point_category", "negative_feedback_category"]
numeric_cols = ["satisfaction_int"]  # 数値特徴量例

# StringIndexer と OneHotEncoder を組み立て
indexers = [StringIndexer(inputCol=c, outputCol=c + "_idx", handleInvalid="keep") for c in categorical_cols]
encoders = [OneHotEncoder(inputCol=c + "_idx", outputCol=c + "_vec") for c in categorical_cols]

feature_cols = [c + "_vec" for c in categorical_cols] + numeric_cols

## 特徴量ベクトル “features” 列を作成（VectorAssembler）

このセルでは index + encoding 出力と数値列をまとめて `features` ベクトルに変換します。

In [0]:
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="keep")

## 前処理とクラスタリングモデルを組み合わせたパイプライン構築

このセルでは、indexer → encoder → assembler → KMeans という流れのパイプラインを構築します。  
クラスタ数 `k` は仮で 3〜5 程度から試してみるのが一般的。

In [0]:
# 例：k = 4 クラスタ
k = 4
kmeans = KMeans(featuresCol="features", predictionCol="cluster", k=k, seed=42)

from pyspark.ml import Pipeline
pipeline = Pipeline(stages=indexers + encoders + [assembler, kmeans])

## パイプライン学習とクラスタ予測

このセルではデータ全体または訓練データを使ってモデル学習を行い、  
各ユーザーにクラスタを割り当てた DataFrame を出力します。

In [0]:
model = pipeline.fit(gold)
clustered = model.transform(gold).select("*", "cluster")
display(clustered.limit(20))

## クラスタ中心とクラスタごとの特徴確認

このセルでは、KMeans モデルからクラスタ中心（centers）を取得し、  
各クラスタの件数・平均満足度などを集計してプロファイルを確認します。

In [0]:
# クラスタ中心
centers = model.stages[-1].clusterCenters()
print("Cluster centers:", centers)

# クラスタごとの集計
cluster_summary = (
    clustered.groupBy("cluster")
    .agg(
        F.count("*").alias("cnt"),
        F.avg("satisfaction_int").alias("avg_satisfaction")
    )
    .orderBy("cluster")
)
display(cluster_summary)

## クラスタごと属性・カテゴリ傾向を可視化

このセルでは、クラスタ毎に年代構成やネガティブカテゴリ出現率を可視化できるような集計を行います。  
例として、クラスタ × negative_feedback_category 件数割合を出します。

In [0]:
# クラスタ × ネガティブカテゴリ 件数割合
cluster_cat = (
    clustered.groupBy("cluster", "negative_feedback_category")
    .agg(F.count("*").alias("cnt"))
)

cluster_tot = clustered.groupBy("cluster").agg(F.count("*").alias("total_cnt"))

cluster_cat_ratio = (
    cluster_cat.join(cluster_tot, on="cluster")
    .withColumn("ratio", F.col("cnt") / F.col("total_cnt"))
    .orderBy("cluster", F.desc("ratio"))
)

display(cluster_cat_ratio)

## ✅ この章のまとめと活用アイデア

このクラスタリングによって得られる洞察例：
- 各クラスタの満足度傾向・カテゴリ傾向・属性傾向が見える  
- 例えば「クラスタ1 は Flavor不満多め・中年男性中心」など仮説抽出  
- マーケティングでターゲット別対応策の仮説を立てやすくなる  

さらに発展させる案：
- 最適なクラスタ数 k を “Elbow 手法” や “シルエットスコア” で見つける  
- 学習済クラスタモデルを新規回答に適用して即座にクラスタ割り当てできるようにする  
- クラスタを特徴量にして、購入意向モデルと併用してパーソナライズ施策立案  

この章によって、ラベルなしでもデータ構造を可視化・仮説化する力を鍛えよう。